<div>
<img src='../../img/WSP_red.png' style='height: 95px; float: left' alt='WSP Logo'/>
<img src='../../img/austroads.png' style='height: 115px; float: right' alt='Client Logo'/>
</div>
<center><h2>AAM6201 Development of Machine-Learning Decision-Support tools for Pavement Asset Management<br>Case Study 1: Project Identification</h2></center>


In [ ]:
# magic command to autoreload changes in src
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from pathlib import Path
from time import time

# Data Preprocessing

This notebook preprocesses a dataset through the following steps in order:

- Apply transformations 
- Adding new columns
- Filtering the dataframe
- Dropping unused columns

As detailed in the configuration

In [ ]:
# load data
from src.nzta_configs.final_config import DATA_DIR
from src.util import load_data
NZTA_DATADIR = Path(DATA_DIR) / 'raw' / 'NZTA' / 'Raw Data'

rutting_skid = load_data(NZTA_DATADIR / 'Rutting_Mean_Skid_Resistance_2016-20.csv')
structure = load_data(NZTA_DATADIR / 'Surface_Structure.csv')
traffic = load_data(NZTA_DATADIR / 'Traffic.csv')
carriageway = load_data(NZTA_DATADIR / 'RAMM_Carriageway.csv')
pavement_layer = load_data(NZTA_DATADIR / 'Pavement_Layer.csv')
deflection_cracking = load_data(NZTA_DATADIR / 'NLTP_Unlimited_dTAGTL.csv')

## Convert columns

In [ ]:
rutting_skid.loc[:, 'Year'] = pd.to_datetime(rutting_skid['Year'].str.split('-', expand=True)[0], format='%Y')
rutting_skid = rutting_skid.replace({'-': np.nan})
structure.loc[:, 'End'] = structure['Start'] + structure['Length']
deflection_cracking.loc[:, 'Name'] = deflection_cracking['Name'].str.split('_', expand=True)[0]
deflection_cracking = deflection_cracking.rename(columns={
    'Name': 'RoadID',
    'From': 'Start',
    'To': 'End'
})

## Drop duplicates

In [ ]:
old_shape = structure.shape[0]
structure_index_cols = ['Road ID', 'Start', 'End', 'Surfacing Date']
structure = structure.drop_duplicates(structure_index_cols)
new_shape = structure.shape[0]

old_shape = rutting_skid.shape[0]
rutting_skid_index_cols = ['Road Name', 'Start', 'End', 'Year']
rutting_skid = rutting_skid.drop_duplicates(rutting_skid_index_cols)
new_shape = rutting_skid.shape[0]

old_shape = traffic.shape[0]
traffic_index_cols = ['Road', 'Start', 'End', 'Count Date']
traffic = traffic.drop_duplicates(traffic_index_cols)
new_shape = traffic.shape[0]

old_shape = carriageway.shape[0]
carriageway_index_cols = ['Road', 'Start', 'End']
carriageway = carriageway.drop_duplicates(carriageway_index_cols)
new_shape = carriageway.shape[0]

old_shape = pavement_layer.shape[0]
pavement_layer_index_cols = ['Road', 'Start', 'End']
pavement_layer = pavement_layer.drop_duplicates(pavement_layer_index_cols, keep='last')
new_shape = pavement_layer.shape[0]

old_shape = deflection_cracking.shape[0]
deflection_cracking_index_cols = ['RoadID', 'Start', 'End']
deflection_cracking = deflection_cracking.drop_duplicates(deflection_cracking_index_cols, keep='last')
new_shape = deflection_cracking.shape[0]

## Drop na index

In [ ]:
old_shape = structure.shape[0]
structure = structure.dropna(subset=structure_index_cols, how='any')
new_shape = structure.shape[0]

old_shape = rutting_skid.shape[0]
rutting_skid = rutting_skid.dropna(subset=rutting_skid_index_cols, how='any')
new_shape = rutting_skid.shape[0]

old_shape = traffic.shape[0]
traffic = traffic.dropna(subset=traffic_index_cols, how='any')
new_shape = traffic.shape[0]

old_shape = carriageway.shape[0]
carriageway = carriageway.dropna(subset=carriageway_index_cols, how='any')
new_shape = carriageway.shape[0]

old_shape = pavement_layer.shape[0]
pavement_layer = pavement_layer.dropna(subset=pavement_layer_index_cols, how='any')
new_shape = pavement_layer.shape[0]

old_shape = deflection_cracking.shape[0]
deflection_cracking = deflection_cracking.dropna(subset=deflection_cracking_index_cols, how='any')
new_shape = deflection_cracking.shape[0]


## Make sure dtypes are correct

In [ ]:
rutting_skid_float_cols = ['ESC', 'ESC RV', 'NAASRA', 'Rutting Mean', 'Wheel Path', 'Left Path Depth', 'Right Path Depth']
rutting_skid.loc[:, rutting_skid_float_cols] = rutting_skid[rutting_skid_float_cols].astype(float)

structure_float_cols = ['Age', 'Width', 'Surface Depth', 'Modified Life', '1st Chip Size', 'Cutter Quantity']
structure_string_cols = ['Surface Material', 'Binder Type']
structure.loc[:, structure_float_cols] = structure[structure_float_cols].astype(float)
structure.loc[:, structure_string_cols] = structure[structure_string_cols].astype(str)

traffic_float_cols = ['ADT', '% Heavy Vehicles', 'VKT', 'ESA per Day', '% Cars', '% LCV', '% MCV', '% HCV I', '% HCV II', '% Bus', 'ESA MCV', 'ESA HCV I', 'ESA HCV II', 'ESA Heavy Vehicles', 'ESA Bus']
traffic_string_cols = ['Carriageway No']
traffic.loc[:, traffic_float_cols] = traffic[traffic_float_cols].astype(float)
traffic.loc[:, traffic_string_cols] = traffic[traffic_string_cols].astype(str)

carriageway_float_cols = ['Width', 'Number of Lanes', 'Lane Width']
carriageway_string_cols = ['Pavement Type', 'Pavement Use', 'Urban/Rural', 'Carriageway No', 'Estimate Loading', 'Travel Direction']
carriageway.loc[:, carriageway_float_cols] = carriageway[carriageway_float_cols].astype(float)
carriageway.loc[:, carriageway_string_cols] = carriageway[carriageway_string_cols].astype(str)

pavement_layer_float_cols = ['Age', 'Life', 'Layer Strength']
pavement_layer_string_cols = ['Layer Material', 'Reconstructed', 'Layer or Subgrade']
pavement_layer.loc[:, pavement_layer_float_cols] = pavement_layer[pavement_layer_float_cols].astype(float)
pavement_layer.loc[:, pavement_layer_string_cols] = pavement_layer[pavement_layer_string_cols].astype(str)

deflection_cracking_float_cols = ['def_fwd', 'Curvature', 'Curvature75', 'crk_alligator']
deflection_cracking.loc[:, deflection_cracking_float_cols] = deflection_cracking[deflection_cracking_float_cols].astype(float)

## Drop unncessary features

In [ ]:
rutting_skid = rutting_skid[rutting_skid_float_cols + rutting_skid_index_cols]
structure = structure[structure_index_cols + structure_float_cols + structure_string_cols]
traffic = traffic[traffic_index_cols + traffic_string_cols + traffic_float_cols]
carriageway = carriageway[carriageway_index_cols + carriageway_float_cols + carriageway_string_cols]
pavement_layer = pavement_layer[pavement_layer_index_cols + pavement_layer_float_cols + pavement_layer_string_cols]
deflection_cracking = deflection_cracking[deflection_cracking_index_cols + deflection_cracking_float_cols]

## Get fixed-length (100m) index

In [ ]:
def cast_to_fixed_section(df, fixed_length: float=100):
    split_df = df.copy()
    split_df = split_df.reset_index(drop=True)

    split_df['Start_Index'] = (split_df['Start'] // fixed_length)
    split_df['End_Index'] = (split_df['End']) // fixed_length - (split_df['End'] % fixed_length == 0)
    split_df.loc[:, 'Duplicate_Count'] = (split_df['End_Index'] - split_df['Start_Index'] + 1) # how many fixed-length sections does this contain?
    split_df = split_df.loc[split_df.index.repeat(split_df['Duplicate_Count'])] # duplicate original section to the number of fixed-length it contains
    split_df = split_df.reset_index().rename(columns={'index': 'position'})

    # assign column matching position with the earliest index with that position
    index_position_lookup = split_df.drop_duplicates(subset=['position'], keep='first')['position'].reset_index()
    split_df = split_df.set_index('position')
    split_df.loc[:, 'original_position'] = index_position_lookup.set_index('position')['index']
    split_df = split_df.reset_index()
    split_df.loc[:, 'fixed_length_Index'] = split_df['Start_Index'] + split_df.index - split_df['original_position']
    split_df = split_df.drop(columns=['position', 'original_position', 'Start_Index', 'End_Index', 'Duplicate_Count'])

    # calculate length contribution
    split_df['fixed_length_Start'] = split_df['fixed_length_Index'] * fixed_length
    split_df['fixed_length_End'] = split_df['fixed_length_Start'] + fixed_length
    split_df['Length_Contribution'] = split_df[['End', 'fixed_length_End']].min(axis=1) - split_df[['Start', 'fixed_length_Start']].max(axis=1)

    # make sure newly minted start ends are used as index instead of the old ones
    split_df.drop(columns=['fixed_length_Index', 'Start', 'End'], inplace=True)
    split_df.rename(columns={'fixed_length_Start': 'Start', 'fixed_length_End': 'End'}, inplace=True)

    return split_df

In [ ]:
def find_weighted_mode(group_df, cat_col):
    all_weights = group_df.groupby([cat_col])[f'weight_{cat_col}'].sum()
    try:
        return all_weights.idxmax()
    except ValueError:
        return np.nan

def group_fixedlength_and_interpolate(df: pd.DataFrame, index_cols: list) -> pd.DataFrame:
    temp_df = df.copy()
    numeric_cols = list(set(temp_df.select_dtypes('number').columns) - set(index_cols).union({'Length_Contribution'}))
    object_cols = list(set(temp_df.select_dtypes('object').columns) - set(index_cols).union({'Length_Contribution'}))

    res_df = temp_df.drop_duplicates(index_cols).set_index(index_cols)
    res_df = res_df.drop(columns=res_df.columns)

    # make weights for original sections when going into the fixed-length section
    na_mask = temp_df[numeric_cols + object_cols].notna().values
    masked_weights = na_mask * temp_df['Length_Contribution'].values.reshape(-1, 1)
    weight_cols = [f'weight_{col}' for col in numeric_cols + object_cols]
    temp_df.loc[:, weight_cols] = masked_weights
    sum_length = temp_df.groupby(index_cols)[weight_cols].transform(np.sum).values
    filtered_weights = masked_weights / np.where(sum_length > 0, sum_length, np.nan)

    # mean
    temp_df.loc[:, numeric_cols] = temp_df[numeric_cols].values * filtered_weights[:, :len(numeric_cols)] 
    res_df.loc[:, numeric_cols] = temp_df.groupby(index_cols)[numeric_cols].sum(min_count=1)

    # mode
    for j, obj_col in enumerate(object_cols):
        sum_weight = temp_df.groupby(index_cols + [obj_col])[f'weight_{obj_col}'].sum()
        res_df.loc[:, obj_col] = sum_weight.reset_index().set_index([obj_col]).groupby(index_cols)[f'weight_{obj_col}'].idxmax()

    del temp_df
    return res_df.reset_index()    

In [ ]:
start = time()
NZTA_DATADIR = Path(DATA_DIR) / 'interim' / 'NZTA'

cut_structure = group_fixedlength_and_interpolate(cast_to_fixed_section(structure, 100), index_cols=structure_index_cols)
cut_structure.to_csv(NZTA_DATADIR / '100m_Surface_Structure_new.csv', index=False)
cut_traffic = group_fixedlength_and_interpolate(cast_to_fixed_section(traffic, 100), index_cols=traffic_index_cols)
cut_traffic.to_csv(NZTA_DATADIR / '100m_Traffic_new.csv', index=False)
cut_rutting_skid = group_fixedlength_and_interpolate(cast_to_fixed_section(rutting_skid, 100), index_cols=rutting_skid_index_cols)
cut_rutting_skid.to_csv(NZTA_DATADIR / '100m_Rutting_Skid_new.csv', index=False)
cut_carriageway = group_fixedlength_and_interpolate(cast_to_fixed_section(carriageway, 100), index_cols=carriageway_index_cols)
cut_carriageway.to_csv(NZTA_DATADIR / '100m_Carriageway_new.csv', index=False)
cut_pavement_layer = group_fixedlength_and_interpolate(cast_to_fixed_section(pavement_layer, 100), index_cols=pavement_layer_index_cols)
cut_pavement_layer.to_csv(NZTA_DATADIR / '100m_Pavement_Layer_new.csv', index=False)
cut_deflection_cracking = group_fixedlength_and_interpolate(cast_to_fixed_section(deflection_cracking, 100), index_cols=deflection_cracking_index_cols)
cut_deflection_cracking.to_csv(NZTA_DATADIR / '100m_Deflection_Cracking_new.csv', index=False)

end = time()
